In [1]:
# set the gpu device
gpu_idx = '3'
import os
from tonic.datasets import NMNIST
import tonic
from tonic import transforms
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_idx

sensor_size = tonic.datasets.NMNIST.sensor_size
# remove isolated events
# sum a period of events into a frame
frame_transform = transforms.Compose([
    transforms.Denoise(filter_time=10000),
    transforms.ToFrame(sensor_size=sensor_size, time_window=1000),
])  # the output of ToFrame is a tuple of (frame, label), where frame is np.ndarray

trainset = tonic.datasets.NMNIST(save_to='/home/zxh/data', train=True, transform=frame_transform)
testset = tonic.datasets.NMNIST(save_to='/home/zxh/data', train=False, transform=frame_transform)
print('trainset size:', len(trainset), 'testset size:', len(testset), 'type of dataset', type(trainset[0][0]), 'shape', trainset[0][0].shape)

trainset size: 60000 testset size: 10000 type of dataset <class 'numpy.ndarray'> shape (298, 2, 34, 34)


In [2]:
import torch
from torch import nn
from spconv import pytorch as spconv  # pay attention to the spconv.pytorch
from conv_models.neurons import LeakyPlain


class ExampleNet(nn.Module):
    def __init__(self, batch_size, u_th, beta, conv_algorithm = spconv.ConvAlgo.Native):
        super(ExampleNet, self).__init__()
        self.batch_size= batch_size
        self.conv_block1 = nn.Sequential(
            spconv.SparseConv2d(2, 16, 5, bias=True, algo=conv_algorithm),
            spconv.SparseBatchNorm(16, eps=1e-5, momentum=0.1),
            spconv.SparseMaxPool2d(2, stride=2),
            LeakyPlain(u_th=u_th, beta=beta, batch_size=batch_size),
        )
        self.conv_block2 = nn.Sequential(
            spconv.SparseConv2d(16, 32, 5, bias=True, algo=conv_algorithm),
            spconv.SparseBatchNorm(32, eps=1e-5, momentum=0.1),
            spconv.SparseMaxPool2d(2, stride=2),
            LeakyPlain(u_th=u_th, beta=beta, batch_size=batch_size),
        )
        self.to_dense = spconv.ToDense() # convert the sparse tensor to dense tensor
        self.flatten = nn.Flatten(start_dim=1)
        self.fc = nn.Sequential(
            nn.Linear(32*5*5, 10),
            LeakyPlain(u_th=u_th, beta=beta, batch_size=batch_size),
        )

    def forward(self, x):
        assert isinstance(x, spconv.SparseConvTensor)
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.to_dense(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = x.view(-1, self.batch_size, 10)  # reshape back into time, batch, *inputs
        return x

batch_size = 128
spconv_net = ExampleNet(batch_size=batch_size, u_th=0.8, beta=0.6).cuda()
print(spconv_net)
for name, param in spconv_net.named_parameters():
    print(name, param.shape)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
import snntorch as snn
from snntorch.surrogate import atan
beta = 0.5
spike_grad = atan()

class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(2, 12, 5)
        self.pool1 = nn.MaxPool2d(2)
        self.snn1 = snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True)
        self.conv2 = nn.Conv2d(12, 32, 5)
        self.pool2 = nn.MaxPool2d(2)
        self.snn2 = snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True)
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(32*5*5, 10)
        self.snn3 = snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.snn1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.snn2(x)
        x = self.flatten(x)
        x = self.linear(x)
        x = self.snn3(x)
        return x

    @staticmethod
    def compute_sparsity(x):
        return (x == 0).float().mean()

snntorch_net = SimpleNet().cuda()
print(snntorch_net)
for name, param in snntorch_net.named_parameters():
    print(name, param.shape)